## Imports

In [ ]:
# Standard Library Imports
import logging
import json
import os

# Third-Party Libraries
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np

# MLflow for Experiment Tracking and Model Management
import mlflow
from mlflow import MlflowClient
from mlflow.types.schema import Schema, ColSpec
from mlflow.types import ParamSchema, ParamSpec
from mlflow.models import ModelSignature

## Define Constants and Paths and Configure Logging

In [ ]:
# Define global experiment and run names to be used throughout the notebook
REGISTER_NAME = "Shakespeare_Model"
EXPERIMENT_NAME = "Shakespeare Text Generation"
RUN_NAME = "Shakespeare_main"


# Set up the paths
DATA_PATH = "shakespeare.txt"
MODEL_PATH = "models/dict_torch_rnn_model.pt"
MODEL_DECODER_PATH = "models/decoder.pt"
MODEL_ENCODER_PATH = "models/encoder.pt"

# Set up the chunk separator for text processing
CHUNK_SEPARATOR = "\n\n"

In [ ]:
# Configure the logging module with desired format and level
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create a logger for this notebook
logger = logging.getLogger('deployment-notebook')
logger.info("Logging configured successfully")

## Get Text Data
This is the text we'll use as a basis for our generations: let's try to generate 'Shakespearean' texts.

This text is from Shakespeare's Sonnet 1. It's one of the 154 sonnets written by William Shakespeare that were first published in 1609. This particular sonnet, like many others, discusses themes of beauty, procreation, and the transient nature of life, urging the beautiful to reproduce so their beauty can live on through their offspring.

In [ ]:
with open(DATA_PATH,'r',encoding='utf8') as f:
    text = f.read()
all_characters = set(text)

## Loading Model

The models are available at the [models](models/) folder, where:
 - [Tensorflow Jupyter Notebook](RNN_for_text_generation_TF.ipynb): `tf_rnn_model.h5`
 - [PyTorch Jupyter Notebook](RNN_for_text_generation_Torch.): `dict_torch_rnn_model.pt`. Also includes the `decoder.pt` and `encoder.pt`

In [ ]:
class CharModel(nn.Module):
    def __init__(self, decoder, encoder, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5, use_gpu=False):
        """Initializes CharModel

        Args:
            decoder: Assigns a unique integer to each character in a dictionary format
            encoder : Reverses the decoder dictionary, providing a mapping from characters to their respective assigned integers.
            all_chars: Set of unique characters found in the text.
            num_hidden: Number of hidden layers. Defaults to 256.
            num_layers: Number of layers. Defaults to 4.
            drop_prob: Regularization technique to prevent overfitting. Defaults to 0.5.
            use_gpu: If the model uses GPU. Defaults to False.
        """
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu
        
        self.all_chars = all_chars
        self.decoder = torch.load(decoder)
        self.encoder = torch.load(encoder)
        
        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
      
    
    def forward(self, x, hidden):
        """_summary_

        Args:
            x (_type_): _description_
            hidden (_type_): _description_

        Returns:
            _type_: _description_
        """
        lstm_output, hidden = self.lstm(x, hidden)       
        drop_output = self.dropout(lstm_output)
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        final_out = self.fc_linear(drop_output)
        
        return final_out, hidden
    
    
    def hidden_state(self, batch_size):
        """_summary_

        Args:
            batch_size (_type_): _description_

        Returns:
            _type_: _description_
        """
        if self.use_gpu:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden))
        
        return hidden


# MLFlow - Register Model

In [ ]:
class RNNModel(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        """_summary_

        Args:
            context (_type_): _description_
        """
        self.model = CharModel(
                        all_chars=all_characters,
                        num_hidden=512,
                        num_layers=3,
                        drop_prob=0.5,
                        use_gpu=False,
                        decoder=context.artifacts['decoder'],
                        encoder=context.artifacts['encoder']
                                           
                    )


        self.model.load_state_dict(torch.load(context.artifacts['model_state_dict']))
        self.model.eval()

    def one_hot_encoder(self, encoded_text, num_uni_chars):
        """
        Convert categorical data into a fixed-size vector of numerical values.

        Args:
            encoded_text: Batch of encoded text.
            num_uni_chars: Number of unique characters

        """
        one_hot = np.zeros((encoded_text.size, num_uni_chars))
        one_hot = one_hot.astype(np.float32)
        one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
        one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
        
        return one_hot

    def predict_next_char(self, char, hidden=None, k=3):
        """_summary_

        Args:
            char (_type_): _description_
            hidden (_type_, optional): _description_. Defaults to None.
            k (int, optional): _description_. Defaults to 3.

        Returns:
            _type_: _description_
        """
        encoded_text = self.model.encoder[char]
        encoded_text = np.array([[encoded_text]])
        encoded_text = self.one_hot_encoder(encoded_text, len(self.model.all_chars))
        inputs = torch.from_numpy(encoded_text)
        inputs = inputs.cpu()
            
        hidden = tuple([state.data for state in hidden])
        lstm_out, hidden = self.model(inputs, hidden)    
        probs = F.softmax(lstm_out, dim=1).data
        probs = probs.cpu()

        
        probs, index_positions = probs.topk(k)        
        index_positions = index_positions.numpy().squeeze()
        probs = probs.numpy().flatten()
        probs = probs/probs.sum()
        char = np.random.choice(index_positions, p=probs)
    
        return self.model.decoder[char], hidden

    def generate_text(self, seed, size, k=3):

        self.model.cpu()
            
        self.model.eval()
        output_chars = [c for c in seed]
        hidden = self.model.hidden_state(1)
        
        for char in seed:
            char, hidden = self.predict_next_char(char, hidden, k=k)
    
        output_chars.append(char)
        for i in range(size):
            char, hidden = self.predict_next_char(output_chars[-1], hidden, k=k)
            output_chars.append(char)
            
        return ''.join(output_chars)
            
        
    def predict(self, context, model_input):
        initial_word = model_input['initial_word'][0]
        size = model_input['size'][0]
        output = self.generate_text(seed=initial_word, size=size)
        
        return output

    @classmethod
    def log_model(cls, model_state_dict, decoder, encoder, demo_folder="demo"): 
        input_schema = Schema(
            [
                ColSpec("string", "initial_word"),
                ColSpec("long", "size")
            ]
        )

        output_schema = Schema(
            [
                ColSpec("string", "generated_text")
            ]
        )
      
        signature = ModelSignature(inputs=input_schema, outputs=output_schema)
             
        requirements = [
            "torch",
            "numpy"
        ]
        mlflow.pyfunc.log_model(
            model_state_dict,
            python_model=cls(),
            artifacts={
                "model_state_dict": model_state_dict, 
                'decoder': decoder, 
                'encoder': encoder, 
                "demo": demo_folder},
            signature=signature,
            pip_requirements=requirements
        )

In [ ]:
mlflow.set_experiment(experiment_name= EXPERIMENT_NAME)

In [ ]:
model_state_dict = MODEL_PATH

In [ ]:
register_name = REGISTER_NAME 

In [ ]:
with mlflow.start_run(run_name = RUN_NAME) as run:
    logger.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    RNNModel.log_model(model_state_dict, MODEL_DECODER_PATH, MODEL_ENCODER_PATH)
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/{model_state_dict}", name=register_name)

In [ ]:
client = mlflow.MlflowClient()
model_metadata = client.get_latest_versions(register_name, stages=["None"])
latest_model_version = model_metadata[0].version
latest_model_version

## Testing registered model

In [ ]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/{register_name}/{latest_model_version}")
print(model.predict({"initial_word": 'Love ', "size": 100}))